In [1]:
%load_ext autoreload
%autoreload 2

from sfsortingresults import SFSortingResults
import spikewidgets as sw

import spikeextractors as se

import os
import yaml

#
from ProcessSorts import ProcessSorts


/home/cat/anaconda3/envs/yass_si_sf/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/cat/anaconda3/envs/yass_si_sf/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


### Instantiate a `SFSortingResults` object

The `SFSortingResults` object allows you to easily retrieve studies, recording names, and available sorted results:

In [2]:
# start kachery daemon
os.system("kachery-p2p start_daemon --label test & ")

# join correct channel
os.system("kachery-p2p join_channel https://gist.githubusercontent.com/magland/0a732d4562ec87d0f1b9baf7eed2718d/raw/spikeforest-download-channel.yaml &")


0

In [3]:
# 
sfresults = SFSortingResults()

# Optional 
# study_names = sfresults.get_study_names()
# print(f"Available studies:\n{study_names}")


Found 5094 sorting outputs


In [7]:
import glob2
import numpy as np

#fnames_sorted_yass = glob2.glob('/media/cat/1TB/spikesorting/sorting_data/recordings_yass_sorted/*.npz')
fnames_sorted_yass = glob2.glob('/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/spikesorting/final_sorts_npz/*.npz')
# 
dir_analysis = '/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/spikesorting/sorting_analysis/'

for fname in fnames_sorted_yass:
    #id_ = 1
    #fname_yass = fnames_sorted_yass[id_]

    p = ProcessSorts(fname,
                     sfresults,
                     dir_analysis)

    if p.study_exists:
        #
        p.extract_all_sorts()

        # 
        p.run_GTstudy()
        

/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/spikesorting/sorting_analysis/
Recording 006_synth has the following sorting outputs:
['IronClust', 'JRClust', 'KiloSort', 'KiloSort2', 'Klusta', 'MountainSort4', 'SpykingCircus', 'Tridesclous']
sorter selected:  IronClust

sorter selected:  JRClust
error in extracting sorter:  'float' object has no attribute 'startswith'

sorter selected:  KiloSort

sorter selected:  KiloSort2

sorter selected:  Klusta

sorter selected:  MountainSort4

sorter selected:  SpykingCircus

sorter selected:  Tridesclous

synth_mearec_neuronexus_noise10_K10_C32 006_synth



/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/spikesorting/sorting_analysis/
Recording rec_16c_1200s_31 has the following sorting outputs:
['HerdingSpikes2', 'IronClust', 'JRClust', 'KiloSort', 'KiloSort2', 'MountainSort4', 'SpykingCircus', 'Tridesclous']
sorter selected:  HerdingSpikes2

sorter selected:  IronClust

sorter selected:  JRClust

sorter selected:  KiloSort

sorter selected:  KiloSort2

s


sorter selected:  MountainSort4

sorter selected:  SpykingCircus

sorter selected:  Tridesclous

synth_mearec_neuronexus_noise10_K10_C32 002_synth



/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/spikesorting/sorting_analysis/
Recording 513_1_1 has the following sorting outputs:
['IronClust', 'JRClust', 'KiloSort', 'KiloSort2', 'Klusta', 'MountainSort4', 'SpykingCircus', 'Tridesclous']
sorter selected:  IronClust

sorter selected:  JRClust

sorter selected:  KiloSort

sorter selected:  KiloSort2

sorter selected:  Klusta

sorter selected:  MountainSort4

sorter selected:  SpykingCircus

sorter selected:  Tridesclous
error in extracting sorter:  Unable to load header of file: /home/cat/kachery-storage/sha1/09/01/ec/0901ec8a1b55195fcc8d4ed35250c1280dbf21d9

paired_boyden32c 513_1_1



/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/spikesorting/sorting_analysis/
Recording 006_synth has the following sorting outputs:
['IronClust', 'JRClust', 'KiloSort', 'KiloSort2', 'Klusta', 'MountainSort4', 'Spy


sorter selected:  MountainSort4

sorter selected:  SpykingCircus

sorter selected:  Tridesclous

synth_mearec_neuronexus_noise10_K10_C32 005_synth



/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/spikesorting/sorting_analysis/
Recording 001_synth has the following sorting outputs:
['IronClust', 'JRClust', 'KiloSort', 'KiloSort2', 'Klusta', 'MountainSort4', 'SpykingCircus', 'Tridesclous']
sorter selected:  IronClust

sorter selected:  JRClust
error in extracting sorter:  'float' object has no attribute 'startswith'

sorter selected:  KiloSort

sorter selected:  KiloSort2

sorter selected:  Klusta

sorter selected:  MountainSort4

sorter selected:  SpykingCircus

sorter selected:  Tridesclous

synth_mearec_neuronexus_noise10_K10_C32 001_synth



/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/spikesorting/sorting_analysis/
Recording 531_2_1 has the following sorting outputs:
['IronClust', 'JRClust', 'KiloSort', 'KiloSort2', 'Klusta', 'MountainSort4', 'SpykingCircus', 'Tridesclous']
sorter selecte


sorter selected:  SpykingCircus

sorter selected:  Tridesclous

paired_boyden32c 915_18_1



/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/spikesorting/sorting_analysis/
Recording m139_200114_195433 has the following sorting outputs:
['HerdingSpikes2', 'IronClust', 'JRClust', 'KiloSort', 'KiloSort2', 'MountainSort4', 'SpykingCircus', 'Tridesclous']
sorter selected:  HerdingSpikes2

sorter selected:  IronClust

sorter selected:  JRClust
error in extracting sorter:  'float' object has no attribute 'startswith'

sorter selected:  KiloSort

sorter selected:  KiloSort2
error in extracting sorter:  'float' object has no attribute 'startswith'

sorter selected:  MountainSort4

sorter selected:  SpykingCircus

sorter selected:  Tridesclous

paired_english m139_200114_195433



/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/spikesorting/sorting_analysis/
Recording 20170622_patch1 has the following sorting outputs:
['HerdingSpikes2', 'IronClust', 'JRClust', 'KiloSort', 'KiloSort2', 'Klusta', 'MountainSo


sorter selected:  SpykingCircus

sorter selected:  Tridesclous

paired_boyden32c 915_8_1



/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/spikesorting/sorting_analysis/
Recording m57_191105_145205 has the following sorting outputs:
['HerdingSpikes2', 'IronClust', 'JRClust', 'KiloSort', 'KiloSort2', 'MountainSort4', 'SpykingCircus', 'Tridesclous']
sorter selected:  HerdingSpikes2

sorter selected:  IronClust

sorter selected:  JRClust
error in extracting sorter:  'float' object has no attribute 'startswith'

sorter selected:  KiloSort

sorter selected:  KiloSort2

sorter selected:  MountainSort4

sorter selected:  SpykingCircus

sorter selected:  Tridesclous

paired_english m57_191105_145205



/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/spikesorting/sorting_analysis/
Recording 419_1_7 has the following sorting outputs:
['IronClust', 'JRClust', 'KiloSort', 'KiloSort2', 'Klusta', 'MountainSort4', 'SpykingCircus', 'Tridesclous']
sorter selected:  IronClust

sorter selected:  JRClust

sorter se

...


sorter selected:  IronClust

sorter selected:  JRClust

sorter selected:  KiloSort

sorter selected:  KiloSort2

sorter selected:  MountainSort4

sorter selected:  SpykingCircus

sorter selected:  Tridesclous

hybrid_drift_siprobe rec_16c_600s_11



/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/spikesorting/sorting_analysis/
Recording rec_64c_1200s_11 has the following sorting outputs:
['HerdingSpikes2', 'IronClust', 'JRClust', 'KiloSort', 'KiloSort2', 'MountainSort4', 'SpykingCircus', 'Tridesclous']
sorter selected:  HerdingSpikes2

sorter selected:  IronClust

sorter selected:  JRClust

sorter selected:  KiloSort

sorter selected:  KiloSort2

sorter selected:  MountainSort4

sorter selected:  SpykingCircus

sorter selected:  Tridesclous

hybrid_static_siprobe rec_64c_1200s_11



/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/spikesorting/sorting_analysis/
Recording 003_synth has the following sorting outputs:
['IronClust', 'JRClust', 'KiloSort', 'KiloSort2', 'Klusta', 'MountainSort4', 'Spyki


sorter selected:  SpykingCircus

sorter selected:  Tridesclous

synth_mearec_neuronexus_noise10_K40_C32 001_synth



/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/spikesorting/sorting_analysis/
Recording rec_32c_600s_31 has the following sorting outputs:
['HerdingSpikes2', 'IronClust', 'JRClust', 'KiloSort', 'KiloSort2', 'MountainSort4', 'SpykingCircus', 'Tridesclous']
sorter selected:  HerdingSpikes2

sorter selected:  IronClust

sorter selected:  JRClust

sorter selected:  KiloSort

sorter selected:  KiloSort2

sorter selected:  MountainSort4

sorter selected:  SpykingCircus

sorter selected:  Tridesclous

hybrid_static_siprobe rec_32c_600s_31



/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/spikesorting/sorting_analysis/
Recording m57_191105_163645 has the following sorting outputs:
['HerdingSpikes2', 'IronClust', 'JRClust', 'KiloSort', 'KiloSort2', 'MountainSort4', 'SpykingCircus', 'Tridesclous']
sorter selected:  HerdingSpikes2

sorter selected:  IronClust

sorter selected:  JRClust
erro


sorter selected:  KiloSort2

sorter selected:  Klusta

sorter selected:  MountainSort4

sorter selected:  SpykingCircus

sorter selected:  Tridesclous

synth_mearec_neuronexus_noise10_K10_C32 004_synth



/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/spikesorting/sorting_analysis/
Recording rec_16c_1200s_31 has the following sorting outputs:
['HerdingSpikes2', 'IronClust', 'JRClust', 'KiloSort', 'KiloSort2', 'MountainSort4', 'SpykingCircus', 'Tridesclous']
sorter selected:  HerdingSpikes2

sorter selected:  IronClust

sorter selected:  JRClust

sorter selected:  KiloSort

sorter selected:  KiloSort2

sorter selected:  MountainSort4

sorter selected:  SpykingCircus

sorter selected:  Tridesclous

hybrid_static_siprobe rec_16c_1200s_31



/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/spikesorting/sorting_analysis/
Recording rec_16c_1200s_11 has the following sorting outputs:
['HerdingSpikes2', 'IronClust', 'JRClust', 'KiloSort', 'KiloSort2', 'MountainSort4', 'SpykingCircus', 'Tridesclous']
sort




/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/spikesorting/sorting_analysis/
Recording 009_synth has the following sorting outputs:
['IronClust', 'JRClust', 'KiloSort', 'KiloSort2', 'Klusta', 'MountainSort4', 'SpykingCircus', 'Tridesclous']
sorter selected:  IronClust

sorter selected:  JRClust
error in extracting sorter:  'float' object has no attribute 'startswith'

sorter selected:  KiloSort

sorter selected:  KiloSort2

sorter selected:  Klusta

sorter selected:  MountainSort4

sorter selected:  SpykingCircus

sorter selected:  Tridesclous

synth_mearec_neuronexus_noise10_K10_C32 009_synth



/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/spikesorting/sorting_analysis/
Recording rec_16c_600s_11 has the following sorting outputs:
['HerdingSpikes2', 'IronClust', 'JRClust', 'KiloSort', 'KiloSort2', 'MountainSort4', 'SpykingCircus', 'Tridesclous']
sorter selected:  HerdingSpikes2

sorter selected:  IronClust

sorter selected:  JRClust

sorter selected:  KiloSort

sorter selected:  KiloSort2



sorter selected:  SpykingCircus

sorter selected:  Tridesclous
error in extracting sorter:  Unable to load header of file: /home/cat/kachery-storage/sha1/09/01/ec/0901ec8a1b55195fcc8d4ed35250c1280dbf21d9

paired_english m57_191105_141906



/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/spikesorting/sorting_analysis/
Recording m139_200114_191955 has the following sorting outputs:
['HerdingSpikes2', 'IronClust', 'JRClust', 'KiloSort', 'KiloSort2', 'MountainSort4', 'SpykingCircus', 'Tridesclous']
sorter selected:  HerdingSpikes2

sorter selected:  IronClust

sorter selected:  JRClust
error in extracting sorter:  'float' object has no attribute 'startswith'

sorter selected:  KiloSort

sorter selected:  KiloSort2

sorter selected:  MountainSort4

sorter selected:  SpykingCircus

sorter selected:  Tridesclous

paired_english m139_200114_191955



/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/spikesorting/sorting_analysis/
Recording 009_synth has the following sorting outputs:
['IronClust', 'JRClus

In [49]:
import pickle
file_to_read = open("/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/spikesorting/sorting_analysis/hybrid_static_siprobe/rec_32c_1200s_11/study.pkl", "rb")

study = pickle.load(file_to_read)
comparisons = study.comparisons
dataframes = study.aggregate_dataframes()

fname_out = '/home/cat/test.npz'
np.savez(fname_out,
        dataframes = dataframes['count_units'])


# dataframes['count_units']



/home/cat/anaconda3/envs/yass_si_sf/lib/python3.6/site-packages/ipykernel_launcher.py:2: ResourceWarning: unclosed file <_io.BufferedReader name='/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/spikesorting/sorting_analysis/hybrid_static_siprobe/rec_32c_1200s_11/study.pkl'>
  


In [50]:
d = dataframes['count_units']

In [51]:

d


,rec_name,sorter_name,num_gt,num_sorter,num_well_detected,num_redundant,num_overmerged,num_false_positive,num_bad
0,rec0,KiloSort,74,45,28,0,1,0,0
1,rec0,yass,74,61,33,0,0,0,0
2,rec0,HerdingSpikes2,74,69,20,0,3,0,0
3,rec0,Tridesclous,74,33,22,0,1,0,0
4,rec0,SpykingCircus,74,49,27,0,0,0,0
5,rec0,JRClust,74,42,24,0,1,0,0
6,rec0,MountainSort4,74,134,21,8,0,52,60
7,rec0,IronClust,74,30,26,0,1,0,0
8,rec0,KiloSort2,74,31,27,0,2,0,0


In [46]:
s1.count_well_detected_units(0.80)

3

In [9]:

# show all studies loaded by 
print(np.sort(sfresults.get_study_names()))

# search for an already sorted yass study
data_yass = np.load('/mnt/adfe6e7b-b77b-4731-bc9e-e639667faba4/spikesorting/final_sorts_npz/LONG_STATIC_600s_16c_010_synth.npz')
target_study = data_yass['study_name']
target_recording = data_yass['rec_name']
sfresults.get_sorting_output_names(target_study,
                                  target_recording)



['hybrid_drift_siprobe' 'hybrid_drift_tetrode' 'hybrid_static_siprobe'
 'hybrid_static_tetrode' 'manual_tetrode_1200s' 'manual_tetrode_2400s'
 'manual_tetrode_600s' 'mea_c30' 'neurocube_quiroga_difficult1'
 'neurocube_quiroga_difficult2' 'neurocube_quiroga_easy1'
 'neurocube_quiroga_easy2' 'neurocube_sim2_11K20' 'neurocube_sim2_2K10'
 'paired_boyden32c' 'paired_crcns' 'paired_english' 'paired_kampff'
 'paired_mea64c' 'paired_monotrode_boyden32c' 'paired_monotrode_crcns'
 'paired_monotrode_kampff' 'paired_monotrode_mea64c' 'synth_bionet_drift'
 'synth_bionet_shuffle' 'synth_bionet_static'
 'synth_magland_noise10_K10_C4' 'synth_magland_noise10_K10_C8'
 'synth_magland_noise10_K20_C4' 'synth_magland_noise10_K20_C8'
 'synth_magland_noise20_K10_C4' 'synth_magland_noise20_K10_C8'
 'synth_magland_noise20_K20_C4' 'synth_magland_noise20_K20_C8'
 'synth_mearec_neuronexus_noise10_K10_C32'
 'synth_mearec_neuronexus_noise10_K20_C32'
 'synth_mearec_neuronexus_noise10_K40_C32'
 'synth_mearec_neuronexu

AssertionError: Study 'LONG_STATIC_600s_16c' not found

In [9]:
#%debug
from spikecomparison import GroundTruthStudy
import glob2
import numpy as np

#
study_folder = '/media/cat/1TB/spikesorting/sorting_analysis/hybrid_static_siprobe/rec_32c_600s_11/'
study = GroundTruthStudy(study_folder)

# fix the ground truth sampling frequency
fnames = glob2.glob(study_folder+ 'sortings/*.npz')
sf = np.load(fnames[0])['sampling_frequency']

gt_data = np.load(study_folder+'/ground_truth/rec0.npz')
np.savez(study_folder+'/ground_truth/rec0.npz',
        unit_ids = gt_data['unit_ids'],
        spike_labels = gt_data['spike_labels'],
        spike_indexes = gt_data['spike_indexes'],
        sampling_frequency = sf)


# 
study.run_comparisons(exhaustive_gt=True, 
                      match_score=0.0,
                      delta_time=3.0)

# 
comparisons = study.comparisons
dataframes = study.aggregate_dataframes()

In [10]:
# comparison summary
dataframes['count_units']

,rec_name,sorter_name,num_gt,num_sorter,num_well_detected,num_redundant,num_overmerged,num_false_positive,num_bad
0,rec0,HerdingSpikes2,74,70,20,0,3,0,0
1,rec0,MountainSort4,74,119,22,7,0,38,45
2,rec0,KiloSort2,74,30,24,0,2,0,0
3,rec0,SpykingCircus,74,41,26,0,1,0,0
4,rec0,KiloSort,74,38,30,0,1,0,0
5,rec0,IronClust,74,32,30,0,0,0,0
6,rec0,JRClust,74,47,24,0,1,0,0
7,rec0,yass,74,64,32,0,0,0,0


In [52]:
data = np.load(study_folder+'/ground_truth/rec0.npz')
print (data['sampling_frequency'])

data = np.load(study_folder+'/sortings/rec0[#]yass.npz')
print (data['sampling_frequency'])


[30000.]
[30000.]


In [ ]:
# MANUAL UNIT MATCHING CODE

from tqdm import trange, tqdm
import numpy as np

def find_nearest_ids(s, times, max_diff):
    
    diff = np.abs(times-s)
    idx = np.where(diff <= max_diff)[0]
    return idx
        
        
def match_unit(gt_spikes, 
               gt_ids,
               times,
               sorted_ids,
               max_diff=100):
    
    n_gt_units = np.unique(gt_ids).shape[0]
    #print ("n units: ", n_gt_units)
    
    ctr=0
    row = np.zeros(n_gt_units,  'int32')
    for s in times:
        idx = find_nearest_ids(s, gt_spikes, max_diff)
#         print (idx)
#         print (gt_ids[idx])
        row[gt_ids[idx]]+=1
        
    return row
        

# 
import glob2
root_dir = '/media/cat/1TB/spikesorting/sorting_analysis/synth_mearec_neuronexus_noise10_K10_C32/004_synth/sortings/'
fnames = glob2.glob(root_dir+'/*.npz')

for fname in fnames:
    print (os.path.split(fname)[1])
    
    #
    fname_gt = os.path.split(fname)[0].replace('sortings','ground_truth/')+'rec0.npz'
    gt_data = np.load(fname_gt)
    gt_times = gt_data['spike_indexes']
    gt_ids = gt_data['spike_labels']-1 # convert to 0-based idnex

    
    # 
    sorted_data = np.load(fname)
    sorted_times = sorted_data['spike_indexes']
    sorted_ids = sorted_data['spike_labels']
    sorted_units = np.unique(sorted_ids)

    # 
    #print ("# of gt spikes: ", gt_ids.shape)
    for unit in sorted_units:
        idx = np.where(sorted_ids==unit)[0]   

        unit_times = sorted_times[idx] 
        #print ("unit times: ", unit_times.shape)

        row = match_unit(gt_times,
                         gt_ids,
                         unit_times,
                         sorted_ids,
                         max_diff = 100)
        print ("ROW: ", row)
        match = row/gt_ids.shape[0]
        best_match = np.argmax(match)
        print ("      match: ", unit, best_match, np.round(match[best_match],3))